In [6]:
import numpy as np
from instrumental import Q_, u
from instrumental import instrument, list_instruments
from instrumental.drivers.daq.ni import Task
from instrumental.drivers.motion.filter_flipper import Position

# import matplotlib.pyplot as plt

In [7]:
paramset = list_instruments()
paramset

C:\Users\elise\miniconda3\lib\site-packages\instrumental_lib-0.8.dev0-py3.10.egg\instrumental\drivers\motion\ecc100.py:24: UserWarning: Driver 'instrumental.drivers.motion.ecc100' is out of date and incompatible with the current Instrumental core. Pull requests fixing this are welcome.
  warnings.warn(


KeyError: 16

In [5]:
FILTER_FLIPPER_TYPE = 37

lib_name = 'Thorlabs.MotionControl.FilterFlipper.dll'

ffi = FFI()
ffi.cdef("""
    typedef struct tagSAFEARRAYBOUND {
      ULONG cElements;
      LONG  lLbound;
    } SAFEARRAYBOUND, *LPSAFEARRAYBOUND;

    typedef struct tagSAFEARRAY {
      USHORT         cDims;
      USHORT         fFeatures;
      ULONG          cbElements;
      ULONG          cLocks;
      PVOID          pvData;
      SAFEARRAYBOUND rgsabound[1];
    } SAFEARRAY, *LPSAFEARRAY;
""")


with open(os.path.join(os.path.dirname(__file__), '_filter_flipper', 'FilterFlipper.h')) as f:
    ffi.cdef(f.read())
lib = ffi.dlopen(lib_name)


NameError: name '__file__' is not defined

In [4]:
class NiceFilterFlipper(NiceLib):
    """Mid-level wrapper for Thorlabs.MotionControl.FilterFlipper.dll"""
    _ffi_ = ffi
    _ffilib_ = lib
    _prefix_ = ('FF_', 'TLI_')
    _buflen_ = 512
    _ret_ = ret_errcheck

    BuildDeviceList = Sig()
    GetDeviceListSize = Sig(ret=ret_return)
    GetDeviceList = Sig('out')
    GetDeviceListByType = Sig('out', 'in')
    GetDeviceInfo = Sig('in', 'out', ret=ret_success)

    # GetDeviceListByTypes seemed not to be properly implemented in the .dll
    #    GetDeviceListByTypes = Sig('out', 'in', 'in')
    GetDeviceListExt = Sig('buf', 'len')
    GetDeviceListByTypeExt = Sig('buf', 'len', 'in')
    GetDeviceListByTypesExt = Sig('buf', 'len', 'in', 'in')

    class Flipper(NiceObject):
        Open = Sig('in')
        Close = Sig('in')
        Identify = Sig('in')
        GetHardwareInfo = Sig('in', 'buf', 'len', 'out', 'out', 'buf', 'len', 'out', 'out', 'out')
        GetFirmwareVersion = Sig('in', ret=ret_return)
        GetSoftwareVersion = Sig('in', ret=ret_return)
        LoadSettings = Sig('in', ret=ret_success)
        PersistSettings = Sig('in', ret=ret_success)
        GetNumberPositions = Sig('in', ret=ret_return)
        Home = Sig('in')
        MoveToPosition = Sig('in', 'in')
        GetPosition = Sig('in', ret=ret_return)
        GetIOSettings = Sig('in', 'out')
        GetTransitTime = Sig('in', ret=ret_return)
        SetTransitTime = Sig('in', 'in')
        RequestStatus = Sig('in')
        GetStatusBits = Sig('in', ret=ret_return)
        StartPolling = Sig('in', 'in', ret=ret_success)
        PollingDuration = Sig('in', ret=ret_return)
        StopPolling = Sig('in')
        RequestSettings = Sig('in')
        ClearMessageQueue = Sig('in')
        RegisterMessageCallback = Sig('in', 'in')
        MessageQueueSize = Sig('in', ret=ret_return)
        GetNextMessage = Sig('in', 'in', 'in', 'in', ret=ret_success)
        WaitForMessage = Sig('in', 'in', 'in', 'in', ret=ret_success)

NameError: name 'ffi' is not defined

In [2]:
def list_instruments():
    NiceFF = NiceFilterFlipper
    NiceFF.BuildDeviceList()
    device_list = to_str(NiceFF.GetDeviceListByTypeExt(FILTER_FLIPPER_TYPE)).split(',')
    return [ParamSet(Filter_Flipper, serial=serial_str)
            for serial_str in device_list
            if serial_str and int(serial_str[:2]) == FILTER_FLIPPER_TYPE]

In [3]:
NiceFF = NiceFilterFlipper

NameError: name 'NiceFilterFlipper' is not defined

In [2]:
from instrumental.conf import user_conf_dir
user_conf_dir

'C:\\Users\\elise\\AppData\\Local\\mabuchilab\\instrumental'

stage.save_instrument('NanoMax_stage')
from instrumental.drivers.motion.BPC203 import BPC203_stage

In [3]:
%cd ..
from experiment_control.srs_microscope.srs_microscope import *

C:\Program Files (x86)\Thorlabs


ModuleNotFoundError: No module named 'experiment_control'

In [ ]:
cam = instrument(paramsets[0]) cam.save_instrument('Thorlabs_camera')

In [4]:
import sys
import clr
import os

sys.path.append(r"C:\Program Files\Thorlabs\Kinesis")
os.getcwd()
clr.AddReference("Thorlabs.MotionControl.DeviceManagerCLI")
clr.AddReference("Thorlabs.MotionControl.FilterFlipperCLI")
from instrumental.drivers.motion.filter_flipper import Position

In [4]:
ff      = instrument("Thorlabs_FilterFlipper",reopen_policy="reuse")

KeyError: 16

In [ ]:
cam = instrument("Thorlabs_camera",reopen_policy="reuse")

ff = instrument(paramsets[2])
ff.save_instrument('Thorlabs_FilterFlipper')

daq = instrument(paramsets[1])
daq.save_instrument('NIDAQ_USB-6259')

daq = instrument('NIDAQ_USB-6259')

In [ ]:
#Configure DAQ input channels
Vi, Vi_str = daq.ai0, 'Dev2/ai0'
#configure DAQ output channels
Vo, Vo_str = daq.ao0, 'Dev2/ao0'

n_samp = 100
fsamp = 1000*u.Hz
test_task = Task(Vi, Vo)
test_task.set_timing(fsamp = fsamp, n_samples=n_samp)


In [ ]:
Vo.write(0*u.V)

In [ ]:
V_data = np.linspace(0,1,n_samp//2)*u.V
V_data = np.concatenate((V_data, V_data[::-1]))
# V_data = np.ones(5)*0*u.V
write_data = {Vo_str: V_data}
read_data = test_task.run(write_data)
test_task.unreserve()
print(read_data)
print(V_data)

# Testing plotting